In [1]:
import sys
print(sys.executable) 
! /Users/lucasyim/opt/anaconda3/bin/python -m pip install utils
! /Users/lucasyim/opt/anaconda3/bin/python -m pip install pretty_midi


/Users/lucasyim/opt/anaconda3/bin/python


In [2]:
'''
This is the data processing script for POP909:A Pop song Dataset for Music Arrangement Generation
============
It will allow you to quickly process the POP909 Files (Midi) into the Google Magenta's music representation 
    as like [Music Transformer](https://magenta.tensorflow.org/music-transformer) 
            [Performance RNN](https://magenta.tensorflow.org/performance-rnn).

'''



import pickle
import os
import sys
import utils
from processor import MidiEventProcessor
import pretty_midi as pyd
import numpy as np



In [65]:
# Create separate folders for the separated midis

for i in range(1,910):
    num = str(i)
    if i < 10:
        num = "00"+num
    elif i < 100:
        num = "0"+num
    os.mkdir(f"../melody/{num}")
    os.mkdir(f"../chords/{num}")


In [4]:
total = 0
def preprocess_midi(path):
    global total
    data = pyd.PrettyMIDI(path)
    
    main_notes = []
    acc_notes = []
    for ins in data.instruments:
        acc_notes.extend(ins.notes)
    for i in range(len(main_notes)):
        main_notes[i].start = round(main_notes[i].start,2)
        main_notes[i].end = round(main_notes[i].end,2)
    for i in range(len(acc_notes)):
        acc_notes[i].start = round(acc_notes[i].start,2)
        acc_notes[i].end = round(acc_notes[i].end,2)
    main_notes.sort(key = lambda x:x.start)
    acc_notes.sort(key = lambda x:x.start)
    mpr = MidiEventProcessor()
    
    # changed from [main_notes, acc_notes] to main_notes+ acc_notes
    repr_seq = mpr.encode(main_notes + acc_notes)
    total += len(repr_seq)
    return repr_seq


def separate_melody(path):
    global total
    data = pyd.PrettyMIDI(path)
    #print(data.instruments)
    
    melody_midi = pyd.PrettyMIDI()
    chords_midi = pyd.PrettyMIDI()
    
    #piano_program = pyd.instrument_name_to_program('PIANO')
    piano = pyd.Instrument(program=0)
    melody = pyd.Instrument(program=0)
    
    
    # go through each instrument and separate into melody and piano
    for instrument in data.instruments: 
        for note in instrument.notes:
            print(note.start)
        if instrument.name == "MELODY":
            melody.notes.extend(instrument.notes)
        if instrument.name == "PIANO":
            piano.notes.extend(instrument.notes)

    melody_midi.instruments.append(melody)
    chords_midi.instruments.append(piano)
    #    print(f"instrument: {instrument}: {len(instrument.notes)}")
    #    print("\n\n\n")
    
    print("../melody/"+path[10:-4]+"melody.mid")
    #melody_midi.write("../melody/"+path[10:-4]+"melody.mid")
    #chords_midi.write("../chords/"+path[10:-4]+"chords.mid")
    
    

def preprocess_pop909_single_midi(midi_root, save_dir):
    save_py = []
    midi_paths = [d for d in os.listdir(midi_root)]
    i = 0
    out_fmt = '{}-{}.data'
    for path in midi_paths:
        #print(' ', end='[{}]'.format(path), flush=True)
        filename = midi_root + path

        if filename[-3:] != "mid":
            continue
        try:
            #data = preprocess_midi(filename)
            print(filename)
            separate_melody(filename)
            
        except KeyboardInterrupt:
            print(' Abort')
            return
        except EOFError:
            print('EOF Error')
            return
        #save_py.append(data)
    #save_py = np.array(save_py)
    #print(save_py.size)
    #np.save("pop909-event-token.npy", save_py)
            
def preprocess_pop909(midi_root, save_dir):
    save_py = []
    midi_paths = [d for d in os.listdir(midi_root)]
    i = 0
    out_fmt = '{}-{}.data'
    for path in midi_paths:
        # print(' ', end='[{}]'.format(path), flush=True)
        
        try: 
            int(path)
        except:
            continue
        filename = midi_root + path + "/" + path + ".mid"
        # print(filename)
        data = []
        if filename[-3:] != "mid":
            continue
        try:
            # data = preprocess_midi(filename)
            
            separate_melody(filename)
        except KeyboardInterrupt:
            print(' Abort')
            return
        except EOFError:
            print('EOF Error')
            return
        #save_py.append(data)
    #save_py = np.array(save_py, dtype=object)
    #print(save_py.size)
    #np.save("pop909-event-token.npy", save_py)
             
# replace the folder with your POP909 data folder
# not sure if each midi file should be its own npy file or if the entire dataset should be put into one file????????
# Basic structure of how they were originally doing it makes me think one file but not sure as of now
# might make more sense to do 1 at a time since we avoid the problem with ragged arrays

# uncomment based on which approach, single_midi for one file (must come up with new name for each file to not overwrite)
# original for all of them in one file.
preprocess_pop909_single_midi("../chords/001/","midi_data/")
#preprocess_pop909("../POP909/", "midi_data/")


../chords/001/001chords.mid
3.0545454545454542
2.7227272727272727
2.8886363636363637
3.2227272727272727
4.222727272727273
4.054545454545455
4.388636363636364
4.554545454545455
5.554545454545455
5.388636363636364
5.888636363636364
5.722727272727273
6.722727272727273
6.888636363636364
7.054545454545454
7.222727272727273
8.222727272727273
8.054545454545455
8.388636363636364
8.554545454545455
9.388636363636364
9.722727272727273
9.388636363636364
9.888636363636364
9.388636363636364
11.222727272727273
10.888636363636364
11.054545454545455
11.722727272727273
10.722727272727273
12.054545454545455
12.054545454545455
12.054545454545455
12.054545454545455
12.388636363636364
12.054545454545455
12.722727272727273
12.722727272727273
12.722727272727273
13.388636363636364
13.388636363636364
13.388636363636364
13.888636363636364
13.722727272727273
14.054545454545455
14.388636363636364
14.388636363636364
15.054545454545455
14.722727272727273
15.388636363636364
15.388636363636364
15.388636363636364
15.72